In [1]:
from lionagi.libs.parse import to_list


```python

def to_list(
    input_: Any,
    /,
    *,
    flatten: bool = False,
    dropna: bool = False,
    unique: bool = False,
    use_values: bool = False,
) -> list:
    """Convert various input types to a list.

    Handles different input types and converts them to a list, with options
    for flattening nested structures and removing None values.

    Args:
        input_: The input to be converted to a list.
        flatten: If True, flattens nested list structures.
        dropna: If True, removes None values from the result.
        unique: If True, returns only unique values (requires flatten=True).
        use_values: If True, uses .values() for dict-like inputs.

    Returns:
        A list derived from the input, processed as specified.

    Raises:
        ValueError: If unique=True and flatten=False.

    Examples:
        >>> to_list(1)
        [1]
        >>> to_list([1, [2, 3]], flatten=True)
        [1, 2, 3]
        >>> to_list([1, None, 2], dropna=True)
        [1, 2]
        >>> to_list({'a': 1, 'b': 2}, use_values=True)
        [1, 2]
    """
```







### 1. NoneType

In [2]:
from pydantic_core import PydanticUndefined
from lionagi.libs.constants import UNDEFINED

print(to_list(None))
print(to_list(PydanticUndefined))
print(to_list(UNDEFINED))

[]
[]
[]


### 1. Basic Types

In [3]:
# boolean
print(to_list(True), to_list(False))

# int float
print(to_list(1), to_list(1.0))

[True] [False]
[1] [1.0]


### 2. String

- str
- bytes
- bytearray

In [4]:
a = "strings"
b = b"strings"
c = bytearray(b"stringarrays")

print(to_list(a))
print(to_list(b))
print(to_list(c))

['strings']
[b'strings']
[bytearray(b'stringarrays')]


In [5]:
## Use Values

print(to_list(a, use_values=True))
print(to_list(b, use_values=True))
print(to_list(c, use_values=True))

['s', 't', 'r', 'i', 'n', 'g', 's']
[115, 116, 114, 105, 110, 103, 115]
[115, 116, 114, 105, 110, 103, 97, 114, 114, 97, 121, 115]


In [6]:
## Unique

print(to_list(a, unique=True, use_values=True))
print(to_list(b, unique=True, use_values=True))
print(to_list(c, unique=True, use_values=True))

['s', 't', 'r', 'i', 'n', 'g']
[115, 116, 114, 105, 110, 103]
[115, 116, 114, 105, 110, 103, 97, 121]


### 3. Enum

In [7]:
from enum import Enum
from lion.libs.constants import UNDEFINED


class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3
    ORANGE = 3
    PURPLE = None
    YELLOW = UNDEFINED


print(to_list(Color))
print(to_list(Color, use_values=True))
print(to_list(Color, unique=True, use_values=True, dropna=True))

[<Color.RED: 1>, <Color.GREEN: 2>, <Color.BLUE: 3>, <Color.PURPLE: None>, <Color.YELLOW: UNDEFINED>]
[1, 2, 3, 3, None, UNDEFINED]
[1, 2, 3]


### 4. Collections

#### 4.1 List / Tuple

In [8]:
# very nested tuple

a = (
    [
        1,
        [2, 3, 4],
        [
            5,
            [(6, 7), UNDEFINED],
            8,
        ],
        [9],
        [5, 6, 7, (8, 9)],
        (10, None),
        1,
    ],
)

print(a)

([1, [2, 3, 4], [5, [(6, 7), UNDEFINED], 8], [9], [5, 6, 7, (8, 9)], (10, None), 1],)


In [9]:
print(to_list(a))
print(to_list(a, flatten=True))
print(to_list(a, unique=True))
print(to_list(a, flatten=True, unique=True, dropna=True))

[[1, [2, 3, 4], [5, [(6, 7), UNDEFINED], 8], [9], [5, 6, 7, (8, 9)], (10, None), 1]]
[1, 2, 3, 4, 5, 6, 7, UNDEFINED, 8, 9, 5, 6, 7, 8, 9, 10, None, 1]
[[1, [2, 3, 4], [5, [(6, 7), UNDEFINED], 8], [9], [5, 6, 7, (8, 9)], (10, None), 1]]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


#### 4.2 Set

- set members are already unique
- collections (list, dict, tuple) are unhashable, so no flatten needed neither

In [10]:
# a complex set
a = {1, 2, 3, 4, 5, 65, None}
print(a)
print(to_list(a))
print(to_list(a, dropna=True))

{None, 1, 2, 3, 4, 5, 65}
[None, 1, 2, 3, 4, 5, 65]
[1, 2, 3, 4, 5, 65]


In [11]:
## Set inside list

b = [[1, 2, 3, 4], {1, 2, 3, 4}, a, None]

print(b)
print(to_list(b))
print(to_list(b, flatten=True))
print(to_list(b, flatten=True, unique=True))
print(to_list(b, flatten=True, unique=True, dropna=True))

[[1, 2, 3, 4], {1, 2, 3, 4}, {None, 1, 2, 3, 4, 5, 65}, None]
[[1, 2, 3, 4], {1, 2, 3, 4}, {None, 1, 2, 3, 4, 5, 65}, None]
[1, 2, 3, 4, 1, 2, 3, 4, None, 1, 2, 3, 4, 5, 65, None]
[1, 2, 3, 4, None, 5, 65]
[1, 2, 3, 4, 5, 65]


#### 4.3 Dict / BaseModel

In [12]:
a = {"a": 1, "b": 2, "g": {"b": 3, "c": 4}}
print(a)

{'a': 1, 'b': 2, 'g': {'b': 3, 'c': 4}}


In [13]:
# dictionary cannot be flattened by the `to_list` function
print(to_list(a, flatten=True))

[{'a': 1, 'b': 2, 'g': {'b': 3, 'c': 4}}]


In [14]:
# similarly, pydantic models cannot be flattened by the `to_list` function
from pydantic import BaseModel


class User(BaseModel):
    name: str
    age: int


a = User(name="John", age=30, friends=[User(name="Jane", age=25)])
print(a)
print(to_list(a))

name='John' age=30
[User(name='John', age=30)]


In [15]:
# but if they are members of a list, the list itself can still behave accordingly

a = User(name="John", age=30)
b = User(name="Jane", age=25)
c = [[a, b], a, b, [[b]], a]

print(len(c))
print(c)

5
[[User(name='John', age=30), User(name='Jane', age=25)], User(name='John', age=30), User(name='Jane', age=25), [[User(name='Jane', age=25)]], User(name='John', age=30)]


In [16]:
print(len(to_list(c, flatten=True)))
print(to_list(c, flatten=True))

6
[User(name='John', age=30), User(name='Jane', age=25), User(name='John', age=30), User(name='Jane', age=25), User(name='Jane', age=25), User(name='John', age=30)]


In [17]:
print(len(to_list(c, flatten=True, unique=True)))
print(to_list(c, flatten=True, unique=True))

2
[User(name='John', age=30), User(name='Jane', age=25)]


#### 4.4 Special Collections

In [18]:
# Named Tuples
from collections import namedtuple

Point = namedtuple("Point", ["x", "y"])
print(p := Point(1, 2))
print(to_list(p))  # → [1, 2]

Point(x=1, y=2)
[1, 2]


In [19]:
# Deque
from collections import deque

dq = deque([1, 2, 3, [3], None])

print(dq)
print(to_list(dq))
print(to_list(dq, flatten=True))
print(to_list(dq, flatten=True, unique=True, dropna=True))

deque([1, 2, 3, [3], None])
[1, 2, 3, [3], None]
[1, 2, 3, 3, None]
[1, 2, 3]


In [20]:
# OrderedDict
from collections import OrderedDict

od = OrderedDict([("a", 1), ("b", 2)])

print(od)
print(to_list(od))
print(to_list(od, use_values=True))

OrderedDict({'a': 1, 'b': 2})
[OrderedDict({'a': 1, 'b': 2})]
[1, 2]


#### 4.5 Others

##### 4.5.1 Generators and higher-order functions

- generator
- range
- filter
- map
- zip

In [21]:
gen = (x for x in range(3))
to_list(gen)

[0, 1, 2]

In [22]:
# Range
to_list(range(3))

[0, 1, 2]

In [23]:
# Filter
to_list(filter(lambda x: x % 2 == 0, range(5)))

[0, 2, 4]

In [24]:
# Map
to_list(map(lambda x: x * 2, range(3)))

[0, 2, 4]

In [25]:
# Zip
to_list(zip([1, 2], ["a", "b"]))

[(1, 'a'), (2, 'b')]

##### 4.5.2 View Objects

In [26]:
d = {"a": 1, "b": 2, "c": 3}
print(d)

{'a': 1, 'b': 2, 'c': 3}


In [27]:
print(to_list(d.keys()))

['a', 'b', 'c']


In [28]:
# these two methods are equivalent
print(to_list(d.values()))
print(to_list(d, use_values=True))

[1, 2, 3]
[1, 2, 3]
